[Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/data)
======

## Data Set

The labeled data set consists of 50,000 IMDB movie reviews, specially selected for sentiment analysis. The sentiment of reviews is binary, meaning the IMDB rating < 5 results in a sentiment score of 0, and rating >=7 have a sentiment score of 1. No individual movie has more than 30 reviews. The 25,000 review labeled training set does not include any of the same movies as the 25,000 review test set. In addition, there are another 50,000 IMDB reviews provided without any rating labels.

## File descriptions

labeledTrainData - The labeled training set. The file is tab-delimited and has a header row followed by 25,000 rows containing an id, sentiment, and text for each review.
## Data fields

* id - Unique ID of each review
* sentiment - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
* review - Text of the review

## Objective
Objective of this dataset is base on **review** we predict **sentiment** (positive or negative) so X is **review** column and y is **sentiment** column

## 1. Load Dataset
we only forcus on "labeledTrainData.csv" file

Let's first of all have a look at the data.

[Click here to download dataset](https://s3-ap-southeast-1.amazonaws.com/ml101-khanhnguyen/week3/assignment/labeledTrainData.tsv)

In [1]:
# Import pandas, numpy
import pandas as pd
import numpy as np

In [2]:
movie_df = pd.read_csv('./labeledTrainData.tsv', sep='\t', encoding='latin-1')
movie_df.sample(10)

,id,sentiment,review
135,11950_2,0,This is without a doubt the worst movie I have...
12153,2555_2,0,This movie is quite possibly one of the most h...
18483,1888_4,0,This movie reminds me of \IrrÃ©versible (2002)...
2302,11785_10,1,I don't quite know how to explain \Darkend Roo...
13344,9761_2,0,It's pretty clear that the director and produc...
15001,8419_1,0,I had been looking forward to seeing Dreamgirl...
2301,10378_2,0,While the premise behind The House Where Evil ...
19957,10342_2,0,Jameson Parker And Marilyn Hassett are the scr...
5770,7774_3,0,"For a good half hour or so, I remember myself ..."
22928,1771_2,0,This has to be one of the worst films I have e...


In [19]:
# Read dataset with extra params sep='\t', encoding="latin-1"


## 2. Preprocessing

In [3]:
# stop words
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ngochieu642/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
# Removing special characters and "trash"
import re
def preprocessor(text):
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text) # Your code here
    
    # Save emoticons for later appending
    # Your code here
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    # Your code here
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    
    return text

In [5]:
print(preprocessor('<h1> I wAs rAiSeD aRoUnD LiOnS :) =('))

 i was raised around lions  :) =(


In [6]:
# tokenizer and stemming
# tokenizer: to break down our twits in individual words
# stemming: reducing a word to its root
from nltk.stem import PorterStemmer
# Your code here

def tokenizer_porter(text):
    porter = PorterStemmer()
    token = [porter.stem(word) for word in text.split()]
    
    return token # Your code here

In [7]:
print(tokenizer_porter('Interesting, i will buy it'))

['interesting,', 'i', 'will', 'buy', 'it']


In [8]:
print(tokenizer_porter('my daught is interested in this product'))

['my', 'daught', 'is', 'interest', 'in', 'thi', 'product']


In [9]:
# split the dataset in train and test
# Your code here
from sklearn.model_selection import train_test_split

X = movie_df['review']
y = movie_df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## 3. Create Model and Train 

Using **Pipeline** to concat **tfidf** step and **LogisticRegression** step

In [11]:
# Import Pipeline, LogisticRegression, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from nltk.corpus import stopwords

stop = stopwords.words('english')

tfidf = TfidfVectorizer(stop_words=stop,
                       tokenizer=tokenizer_porter,
                       preprocessor=preprocessor) # Your code here

clf = Pipeline([('vect', tfidf),
                ('clf', LogisticRegression(random_state=0))])

clf.fit(X_train, y_train)

/home/ngochieu642/.virtualenvs/dataScience/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/home/ngochieu642/.virtualenvs/dataScience/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=<function preprocessor at 0x7f1a72c839d8>,
                                 smooth_idf=True,
                                 stop_words=['i', 'me', 'my', 'myself', '...
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenizer_porter at 0x7f1a72c83b70>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
         

## 4. Evaluate Model

In [12]:
# Using Test dataset to evaluate model
# classification_report
# confusion matrix
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Now apply those above metrics to evaluate your model
# Your code here
prediction = clf.predict(X_test)
print(accuracy_score(y_test, prediction))
print(confusion_matrix(y_test, prediction))
print(classification_report(y_test, prediction))

0.8824
[[3264  522]
 [ 360 3354]]
              precision    recall  f1-score   support

           0       0.90      0.86      0.88      3786
           1       0.87      0.90      0.88      3714

    accuracy                           0.88      7500
   macro avg       0.88      0.88      0.88      7500
weighted avg       0.88      0.88      0.88      7500



## 5. Export Model 

In [15]:
# Using pickle to export our trained model
import pickle
import os
pickle.dump(clf, open(os.path.join('data', 'movie-logisticRegression.pkl'), 'wb'), protocol=4)

## 6. Import Model

In [21]:
with open('./data/movie-logisticRegression.pkl','rb') as f:
    clf_2 = pickle.load(f)

In [22]:
clf_2.predict(X_test)

/home/ngochieu642/.virtualenvs/dataScience/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


array([1, 1, 1, ..., 1, 1, 0])

In [23]:
print(accuracy_score(y_test, prediction))
print(confusion_matrix(y_test, prediction))
print(classification_report(y_test, prediction))

0.8824
[[3264  522]
 [ 360 3354]]
              precision    recall  f1-score   support

           0       0.90      0.86      0.88      3786
           1       0.87      0.90      0.88      3714

    accuracy                           0.88      7500
   macro avg       0.88      0.88      0.88      7500
weighted avg       0.88      0.88      0.88      7500

